In [2]:
import ee

ModuleNotFoundError: No module named '_curses'

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account, 
    key_file="D:\\mehdi\\WaterBalance_new\\notebook\\private-key.json"
)

ee.Initialize(credentials)